# Movie Classification

Oftentimes, we arbitrarily categorize things based on a general vibe. Is there anything more substantial to the subdivisions of movie eras beyond a general aesthetic? Can we reliably distinguish the eras of movie making based solely on the posters or movie stills? I want to explore the aesthetics of different movie eras while learning new tools like PyTorch and Python Image Library for image classification and exploration. 

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from collections import Counter
from tqdm import tqdm

# for image exploration
from easyimages import EasyImageList
from PIL import Image

# torch
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

<h2><u>Data Exploration</u></h2>

<h3>Metadata</h3>

In [2]:
df = pd.read_csv('./data/metadata.csv')
df.head()

,filename,title,release_year,genres
0,346.jpg,Seven Samurai,1954,"[28, 18]"
1,567.jpg,Rear Window,1954,"[53, 9648]"
2,173.jpg,"20,000 Leagues Under the Sea",1954,"[12, 878, 10751]"
3,43332.jpg,Demetrius and the Gladiators,1954,"[12, 36, 18]"
4,10178.jpg,The Caine Mutiny,1954,"[18, 10752]"


In [3]:
# should be no invalid data because of the way we pulled it, but checking anyway
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6063 entries, 0 to 6062
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   filename      6063 non-null   object
 1   title         6063 non-null   object
 2   release_year  6063 non-null   int64 
 3   genres        6063 non-null   object
dtypes: int64(1), object(3)
memory usage: 189.6+ KB


In [4]:
plt.hist(df['release_year'], edgecolor='black', linewidth=0.4)
plt.title("Distribution of Movies by Year")
plt.xlabel("Year")
plt.ylabel("Num Movies")
plt.tight_layout()
plt.show()

/var/folders/vq/1vm3d1qd40dc31fh9jl7k1fc0000gn/T/ipykernel_31197/383055285.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Kind of as expected, TMDB has much better capabilities for more recent and popular years, and some earlier years will have substantially less movies than later ones. We will try to account for this in our era subdivisions, but overall, we should be ok.

In [5]:
# Let's also make sure each entry in the csv has a corresponding image
res = []
for filename in df['filename']:
    if not os.path.exists(os.path.join("/Users/anubis/Desktop/Personal_Projects/movie-decade-classification/data/posters/", filename)):
        res.append(filename)
res

[]

##### Defining Film Eras:
Please check the <a href='https://github.com/Anubis-vc/movie-era-classification/blob/main/README.md'>README</a> for an in-depth discussion on how I chose these eras. As a reminder, I've attached the eras below:

1. *The Golden Age of Hollywood (1939 - 1949)*

2. *Post-War to New Hollywood Transition (1950 - 1966)*

3. *New Hollywood (1967 - 1982)*

4. *Blockbuster Era (1983 - 1994)*

5. *Indie and Digital Era (1995 - 2006)*

6. *Streaming and Superhero Era (2007 - 2017)*

7. *Modern Era/Post-Streaming (2018 - Current)* 

In [6]:
df.sort_values(by=['release_year'], inplace=True)
conditions = [
	df['release_year'] < 1950,
    df['release_year'] < 1967,
	df['release_year']< 1983,
 	df['release_year']< 1995,
    df['release_year']<  2007,
    df['release_year']< 2018
]

eras = ["Golden Age", "Post-War Transition", "New Hollywood", "Blockbuster Era", "Digital Era", "Franchise Era"]

df['era'] = np.select(conditions, eras, default="Current Era")
df


,filename,title,release_year,genres,era
277,36488.jpg,Bachelor Mother,1939,"[35, 10749]",Golden Age
286,1504381.jpg,Brazil: 1st Part,1939,[],Golden Age
285,1504385.jpg,Brazil: 2nd Part,1939,[99],Golden Age
270,630.jpg,The Wizard of Oz,1939,"[12, 14, 10751]",Golden Age
283,1511845.jpg,Sitara,1939,[],Golden Age
...,...,...,...,...,...
5798,1049082.jpg,Jumbo,2025,"[16, 35, 18, 10751, 14]",Current Era
5799,1380393.jpg,The Chosen: Last Supper Part 1,2025,[18],Current Era
5800,1257960.jpg,Sikandar,2025,"[28, 18]",Current Era
5782,1139695.jpg,She's Got No Name,2025,"[18, 80]",Current Era


In [7]:
plt.figure(figsize=(7, 4))
sns.countplot(data=df, x='era', edgecolor='black', linewidth=0.4)
plt.title("Distribution of Movies by Era")
plt.xlabel("Era")
plt.ylabel("Num Movies")
plt.xticks(rotation=45, ha='right')
plt.show()

/var/folders/vq/1vm3d1qd40dc31fh9jl7k1fc0000gn/T/ipykernel_31197/718476083.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
# let's check for duplicate titles in case
df.sort_values(by='release_year')
duplicated = df.duplicated(subset=['title'])
print(np.sum(duplicated))

# by default, grabbing the latest duplicate title
latest_dups = df[duplicated]
latest_dups

154


,filename,title,release_year,genres,era
125,1580.jpg,Rope,1948,"[53, 80, 18]",Golden Age
58,1506593.jpg,Rebecca,1950,[10749],Post-War Transition
543,1426786.jpg,USSR Air Fleet Day,1951,[99],Post-War Transition
224,1426806.jpg,USSR Air Fleet Day,1952,[99],Post-War Transition
880,1484868.jpg,To Catch a Thief,1958,"[18, 80]",Post-War Transition
...,...,...,...,...,...
5745,552524.jpg,Lilo & Stitch,2025,"[10751, 878, 35, 12]",Current Era
5749,1087192.jpg,How to Train Your Dragon,2025,"[14, 10751, 28]",Current Era
5754,1061474.jpg,Superman,2025,"[878, 12, 28]",Current Era
5814,1239655.jpg,Friendship,2025,[35],Current Era


It seems like these duplicates come from remakes sharing the same name. We can check this easily by taking a subset of title and year instead. Just for fun, let's also see when most of these remakes are from.

In [9]:
np.sum(df.duplicated(subset=['title', 'release_year']))

0

In [10]:
sns.countplot(data=latest_dups, x='era', edgecolor='black', linewidth=0.4)
plt.title("Remakes by Era")
plt.xlabel("Era")
plt.ylabel("Num Remakes")
plt.xticks(rotation=45, ha='right')
plt.show()

/var/folders/vq/1vm3d1qd40dc31fh9jl7k1fc0000gn/T/ipykernel_31197/1174242371.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
# since total counts are different, let's find proportions instead
era_order = df['era'].drop_duplicates().tolist()
era_counts = df['era'].value_counts().reindex(era_order)
remake_counts = latest_dups['era'].value_counts().reindex(era_order)

proportions = (remake_counts / era_counts)
proportions

era
Golden Age             0.002020
Post-War Transition    0.006173
New Hollywood          0.013805
Blockbuster Era        0.013542
Digital Era            0.044792
Franchise Era          0.050057
Current Era            0.048589
Name: count, dtype: float64

In [12]:
proportions.plot(kind='bar',
                 edgecolor='black',
                 linewidth=0.4,
                 xlabel='Era',
                 ylabel='Proportion',
                 title='Proportion of Remakes by Era')
plt.xticks(rotation=45, ha='right')
plt.show()

/var/folders/vq/1vm3d1qd40dc31fh9jl7k1fc0000gn/T/ipykernel_31197/3058240044.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


So clearly, the prevalence of remakes is something recent which does make sense: you have to have something to remake before you can remake it. That concludes some preliminary transformation of the metadata.

<h3>Images</h3>

It's much easier to explore numerical/categorical data than it is images, but the set of images is our entire mapping domain, so it would be cursory to skip exploring the image data for our problem.

First, let's get a feel for the images in general; what do they look like and are there any obvious black screens or blurry images?

In [13]:
list = EasyImageList.from_folder('/Users/anubis/Desktop/Personal_Projects/movie-decade-classification/data/posters')
list.symlink_images()
list.html(sample=200)

Ok, pretty good! From this random sample, it looks like most of our images came out well. Looking at these images from here I'm having a hard time distinguishing either era or genre from any of the posters just looking at the images. Hopefully the network finds a better mapping.

Let's see what else we can find out, like the sizes of the images. TheMovieDatabase API documentation can be a little hard to parse for image data, so we can figure out the size manually for each poster. A really important new idea I came accross when exploring image data is anchor boxes. The best resource I found is <a href='https://d2l.ai/chapter_computer-vision/anchor.html'>here</a>, but I will summarize below.

Anchor boxes - Your neural network will sample different regions of your images to try and determine features of interest. The article above uses the term scale a lot, this is *not* the definition of scale used in photography. Instead, scale refers to the size of the anchor box as compared to the image. The network will produce many bounding boxes of different aspect ratios and sizes anchored at each pixel of the image. After a lot of trial an error, the network ideally figures out the perfect bounding boxes for the important features of your image. Usually, on image analysis you'll try to find the aspect ratio and area of these ideal bounding boxes for your image.

Luckily, our object is the entire image, so we can go ahead and do a raw analysis of the image dimensions.

In [14]:
POSTER_DIR_FINAL = '/Users/anubis/Desktop/Personal_Projects/movie-decade-classification/data/posters/'
img_dim_data = []

for img_name in os.listdir(path=POSTER_DIR_FINAL):
    img_path = os.path.join(POSTER_DIR_FINAL, img_name)
    with Image.open(img_path) as img:
        width, height = img.size
        aspect_ratio = width / height
        area = width * height
        img_dim_data.append({
            'name': img_name,
            'height': height,
            'width': width,
            'aspect_ratio': aspect_ratio,
            'area': area,
            'format': img.format,
            'mode': img.mode
        })

img_dims_df = pd.DataFrame(img_dim_data)
print(f"Total Images: {len(img_dims_df)}")
print(f"Unique dims: {len(img_dims_df[['height', 'width']].drop_duplicates())}")
print(f"Unique aspect ratios: {len(img_dims_df['aspect_ratio'].drop_duplicates())}")
print(f"Unique areas: {len(img_dims_df['area'].drop_duplicates())}\n")
print(img_dims_df[['height', 'width', 'aspect_ratio', 'area']].describe())

Total Images: 6063
Unique dims: 38
Unique aspect ratios: 38
Unique areas: 38

            height   width  aspect_ratio          area
count  6063.000000  6063.0   6063.000000   6063.000000
mean    230.219528   154.0      0.669121  35453.807356
std       3.758830     0.0      0.011934    578.859834
min     202.000000   154.0      0.626016  31108.000000
25%     231.000000   154.0      0.666667  35574.000000
50%     231.000000   154.0      0.666667  35574.000000
75%     231.000000   154.0      0.666667  35574.000000
max     246.000000   154.0      0.762376  37884.000000


In [15]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# scatter plot for height vs. width
axes[0].scatter(img_dims_df['width'], img_dims_df['height'], alpha=0.6)
axes[0].set_xlabel('Width (pixels)')
axes[0].set_ylabel('Height (pixels)')
axes[0].set_title('Image Dimensions')
axes[0].grid(True, alpha=0.3)

# histogram for area
axes[1].hist(img_dims_df['area'], alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Area (pixels)')
axes[1].set_ylabel('Count')
axes[1].set_title('Image Area Distribution')
axes[1].grid(True, alpha=0.3)

# histogram for aspect ratio
axes[2].hist(img_dims_df['aspect_ratio'], alpha=0.7, edgecolor='black')
axes[2].set_xlabel('Aspect Ratio (width/height)')
axes[2].set_ylabel('Count')
axes[2].set_title('Aspect Ratio Distribution')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

/var/folders/vq/1vm3d1qd40dc31fh9jl7k1fc0000gn/T/ipykernel_31197/3696807581.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


So it looks like all the widths are the same with differing heights. The areas and aspect ratios have a few outliers we'll have to adjust before moving on so all our images are the same size. We'll take care of that during training.

When I can, I will try to modify the images by cropping and adding padding such that the aspect ratio of the internal image stays the same, but the overall image changes aspect ratio for uniformity. This is called a non-destructive resize. Additionally, I want all my images to have the same pixel values. Since most of the posters are overwhelmingly of size 154x231, that's what I will use.

Additionally, I'd like to explore the color distributions on a few sample images to get an idea of what we see predominantly.

In [16]:
sample_img = '/Users/anubis/Desktop/Personal_Projects/movie-decade-classification/data/posters/1084736.jpg'
with Image.open(sample_img) as img:
    # convert image to numpy array
    img_array = np.array(img)
    
    # find mean of red channel
    mean_r_value = np.mean(img_array[:, :, 0])
    # find mean of green channel
    mean_g_value = np.mean(img_array[:, :, 1])
    # find mean of blue channel
    mean_b_value = np.mean(img_array[:, :, 2])
    # find overall mean of image
    mean_value = np.mean(img_array)
    
    print(mean_r_value)
    print(mean_g_value)
    print(mean_b_value)
    print(mean_value)
    

116.09585652442796
109.58708607409906
97.73432844212066
107.80575701354923


Each of the first three values computes the mean of a RGB color channel (0-255). A higher value means a greater pprevalence of that color. The final value is the overall brightness of the image, a lower value corresponds to a darker image.

Let's see how the images look in general. I want to use the values we find here to compute a mean and standard deviation. Usually, people will divide each channel by 255 to center the values between 0 and 1, but we can get even better results by customizing this transformation to our dataset.

In [17]:
mean_r, mean_g, mean_b, mean_all = [], [], [], []
all_img_names = os.listdir(POSTER_DIR_FINAL)
random_indexes = np.random.randint(0, len(all_img_names), size=1000)
for i in random_indexes:
    img_path = os.path.join(POSTER_DIR_FINAL, all_img_names[i])
    
    with Image.open(img_path) as img:
        img = img.convert('RGB')
        arr = np.array(img)
        mean_r.append(np.mean(arr[:, :, 0]))
        mean_g.append(np.mean(arr[:, :, 1]))
        mean_b.append(np.mean(arr[:, :, 2]))
        mean_all.append(np.mean(arr))

plt.figure(figsize=(12, 6))
sns.histplot(mean_r, color='r', label='Red', kde=True)
sns.histplot(mean_g, color='g', label='Green', kde=True)
sns.histplot(mean_b, color='b', label='Blue', kde=True)
plt.title("Distribution of Mean Pixel Values (RGB Channels)")
plt.xlabel("Mean Value (0-255)")
plt.ylabel("Frequency")
plt.legend()
plt.show()


/var/folders/vq/1vm3d1qd40dc31fh9jl7k1fc0000gn/T/ipykernel_31197/2468636279.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [19]:
# for transformation later, we have to compute the overall mean by channel instead of the individual imgs
channel_sum = torch.zeros(3)
channel_sq_sum = torch.zeros(3)
num_pixels = 0

# need to transform the images before computing the stats, this caused an error later where normalization
# did not occur properly because the mean and std were different after images are resized
stats_transforms = transforms.Compose([
    transforms.Resize((231, 154)),
    transforms.ToTensor(),
])

for img_name in tqdm(all_img_names):
    img_path = os.path.join(POSTER_DIR_FINAL, img_name)
    
    with Image.open(img_path).convert('RGB') as img:
        # apply same preprocessing as later
        img_array = stats_transforms(img) # shape (3, 231, 154)
        
        channel_sum += img_array.sum(dim=(1, 2))
        channel_sq_sum += (img_array ** 2).sum(dim=(1, 2))
        num_pixels += img_array.shape[1] * img_array.shape[2]

mean = channel_sum / num_pixels
# using formula Var(X) = E[X^2] - (E[X])^2
std = torch.sqrt(channel_sq_sum / num_pixels - mean ** 2)

print(f"Dataset Mean (R, G, B):\t{mean}")
print(f"Dataset Std (R, G, B):\t{std}")

100%|██████████| 6063/6063 [00:05<00:00, 1076.51it/s]

Dataset Mean (R, G, B):	tensor([0.4849, 0.4309, 0.4019])
Dataset Std (R, G, B):	tensor([0.3589, 0.3404, 0.3360])


## Model Creation

### Create Torch Dataset from Directory

First creating a custom dataset class as referenced by the [torch manuals](https://docs.pytorch.org/tutorials/beginner/data_loading_tutorial.html)

In [20]:
class PosterDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None, useEras=True):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        self.labels2idx = { label: idx for idx, label in enumerate(self.df['era'].unique()) } if useEras else {}
        self.idx2label = { idx: label for label, idx in self.labels2idx.items() }
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name = self.df.loc[index, 'filename']
        label = self.df.loc[index, 'era']
        label_index = self.labels2idx[label]
        
        full_img_path = os.path.join(self.image_dir, img_name)
        img = Image.open(full_img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(img)
        
        return { 'image':image, 'label':label_index }
        

In [22]:
image_transforms = transforms.Compose([
	transforms.Resize((231, 154)),
	# to tensor automatically perfroms the / 255
	transforms.ToTensor(),
	# now we can use the mean and std dev computed early to normalize into the range [-1, 1]
	transforms.Normalize(mean=mean, std=std),
])

dataset = PosterDataset(df[['filename', 'era']], image_dir=POSTER_DIR_FINAL, transform=image_transforms)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_ds, test_ds = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=16)

for i, sample in enumerate(train_loader):
    print(f"{i}\tImg Tensor Shape: {sample['image'].shape}")
    
    if i == 5:
        break

0	Img Tensor Shape: torch.Size([16, 3, 231, 154])
1	Img Tensor Shape: torch.Size([16, 3, 231, 154])
2	Img Tensor Shape: torch.Size([16, 3, 231, 154])
3	Img Tensor Shape: torch.Size([16, 3, 231, 154])
4	Img Tensor Shape: torch.Size([16, 3, 231, 154])
5	Img Tensor Shape: torch.Size([16, 3, 231, 154])


## Model Definition